In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [6]:
class Config(object):
    def __init__(self, **entries):
        self.__dict__.update(entries)

    layer_num = 1
    hidden_size = 100
    learning_rate = 1e-3
    cell_type = 'GRU'
    lamda = 1
    D_epoch = 1
    GPU = '0'
    '''User defined'''
    batch_size = None  # batch_size for train
    epoch = None  # epoch for train
    lamda_D = None  # epoch for training of Discriminator
    G_epoch = None  # epoch for training of Generator
    train_data_filename = None
    test_data_filename = None
    save = None
    verbose = 2

In [5]:
from utils import MISSING_VALUE
def RNNCell(type, hidden_size):
    if type == 'LSTM':
        cell = layers.LSTMCell(hidden_size)
    elif type == 'GRU':
        cell = layers.GRUCell(hidden_size)
    return cell

In [2]:
cell = layers.GRU(10)

Metal device set to: 

2022-10-04 14:25:24.790100: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-04 14:25:24.791183: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [3]:
cell(tf.ones((8,5)))

ValueError: Input 0 of layer "gru" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (8, 5)

In [4]:
# inputs (20, 54, 16)
# outputs (1060, 16), (20, 100)
hidden_size = 100
batch_size = 20
num_steps = 54
input_dimension_size = 16

class G2(keras.Sequential):
  def __init__(self, *args, **kwargs):
    super().__init__(name='G2', *args, **kwargs)
    self.add(keras.Input((num_steps, input_dimension_size)))
    self.add(layers.GRU(hidden_size, return_state=True))
    

In [28]:
g2 = layers.GRU(hidden_size, return_state=True, return_sequences=True, stateful=True)
inputs = tf.ones((batch_size, num_steps, input_dimension_size))
sequences, state = g2(inputs)

In [17]:
#state = g2.get_initial_state(inputs)

In [27]:
g2(inputs[:, 0, :])

ValueError: Input 0 of layer "gru_4" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (20, 16)

In [29]:
prediction_target_hidden_output = sequences#[:-1]
tf.reshape(tensor=tf.concat(values=prediction_target_hidden_output, axis=1), shape=[-1, hidden_size])

<tf.Tensor: shape=(1080, 100), dtype=float32, numpy=
array([[ 0.09465797,  0.11787497,  0.15286814, ..., -0.02413253,
         0.36121944,  0.02741932],
       [ 0.16339351,  0.18988925,  0.18446703, ..., -0.05323631,
         0.48296183,  0.05477516],
       [ 0.20765798,  0.22633107,  0.18031006, ..., -0.07515536,
         0.5225322 ,  0.07717809],
       ...,
       [ 0.2734821 ,  0.25643003,  0.16249922, ..., -0.11938749,
         0.54036826,  0.12415558],
       [ 0.2734821 ,  0.25643003,  0.16249922, ..., -0.11938749,
         0.54036826,  0.12415558],
       [ 0.2734821 ,  0.25643003,  0.16249922, ..., -0.11938749,
         0.54036826,  0.12415558]], dtype=float32)>

In [28]:
prediction_target_hidden_output

<tf.Tensor: shape=(19, 54, 100), dtype=float32, numpy=
array([[[ 0.0213209 , -0.2767173 ,  0.1447619 , ..., -0.00327762,
          0.11568137, -0.02670568],
        [ 0.0253525 , -0.3969433 ,  0.24228829, ..., -0.02031945,
          0.19410197, -0.0963363 ],
        [ 0.02189449, -0.4503873 ,  0.29757336, ..., -0.03708171,
          0.24474704, -0.15989162],
        ...,
        [ 0.01442947, -0.49457717,  0.37103027, ..., -0.08089819,
          0.31733423, -0.26936498],
        [ 0.01442947, -0.49457717,  0.37103027, ..., -0.08089819,
          0.31733423, -0.26936498],
        [ 0.01442947, -0.49457717,  0.37103027, ..., -0.08089819,
          0.31733423, -0.26936498]],

       [[ 0.0213209 , -0.2767173 ,  0.1447619 , ..., -0.00327762,
          0.11568137, -0.02670568],
        [ 0.0253525 , -0.3969433 ,  0.24228829, ..., -0.02031945,
          0.19410197, -0.0963363 ],
        [ 0.02189449, -0.4503873 ,  0.29757336, ..., -0.03708171,
          0.24474704, -0.15989162],
        ...,

In [29]:
inputs = tf.ones((20,54,16))

In [31]:
for i in range(10):
  if i > 0:
    print("i>0")
  if i == 0:
    print("i==0")
  else:
    print("else")

i==0
i>0
else
i>0
else
i>0
else
i>0
else
i>0
else
i>0
else
i>0
else
i>0
else
i>0
else


In [60]:
class Generator(layers.Layer):

    def __init__(self, config, *args, **kwargs):
        super().__init__(name='Generator_LSTM', *args, **kwargs, )
        self.batch_size = config.batch_size  # configurable
        self.hidden_size = config.hidden_size  # congigurable for GRU/LSTM
        self.num_steps = config.num_steps  # length of input array
        # dimension of input eleemnt array univariate 1
        self.input_dimension_size = config.input_dimension_size
        self.cell_type = config.cell_type  # RNN Cell type
        self.lamda = config.lamda  # coefficient that balances the prediction loss
        self.class_num = config.class_num  # number of targes
        self.layer_num = config.layer_num  # number of layers of AJRNN

        with tf.compat.v1.variable_scope(self.name):
            # project layer weight W and bias
            self.W = tf.Variable(tf.random.truncated_normal(
                [self.hidden_size, self.input_dimension_size], stddev=0.1), dtype=tf.float32, name='Project_W')
            self.bias = tf.Variable(tf.constant(
                0.1, shape=[self.input_dimension_size]), dtype=tf.float32, name='Project_bias')
            # lstm_keep_prob, classfication_keep_prob = 1.0, 1.0
            # construct cells with the specific layer_num
            self.mulrnn_cell = tf.keras.layers.StackedRNNCells(
                [RNNCell(type=self.cell_type, hidden_size=self.hidden_size) for _ in range(self.layer_num)]
            )

        self.dense1 = layers.Dense(self.class_num)
    
    @tf.compat.v1.keras.utils.track_tf1_style_variables
    def __call__(self, input): # , prediction_target, mask, label_target):

        with tf.compat.v1.variable_scope(self.name):

            # initialize state to zero
            init_state = self.mulrnn_cell.get_initial_state(batch_size=self.batch_size, dtype=tf.float32)
            state = init_state

            outputs = list()

            # makes cell run
            # outputs has list of 'num_steps' with each element's shape (batch_size, hidden_size)
            with tf.compat.v1.variable_scope("RNN"):
                for time_step in range(self.num_steps):
                    if time_step > 0:
                        tf.compat.v1.get_variable_scope().reuse_variables()

                    if time_step == 0:
                        (cell_output, state) = self.mulrnn_cell(input[:, time_step, :], state)
                        outputs.append(cell_output)

                    else:
                        # comparison has shape (batch_size, self.input_dimension_size) with elements 1 (means missing) when equal or 0 (not missing) otherwise
                        comparison = tf.equal(input[:, time_step, :], tf.constant(MISSING_VALUE))
                        current_prediction_output = tf.matmul(outputs[time_step - 1], self.W) + self.bias
                        # change the current_input, select current_prediction_output when 1 (missing) or use input when 0 (not missing)
                        current_input = tf.where(comparison, current_prediction_output, input[:, time_step, :])
                        (cell_output, state) = self.mulrnn_cell(current_input, state)
                        outputs.append(cell_output)

            # last_cell has the last_time_step of shape (batch_size, hidden_size)
            last_cell = outputs[-1]

            # prediction_target_hidden_output has list of 'num_steps - 1' with each element's shape (batch_size, hidden_size)
            prediction_target_hidden_output = outputs[:-1]

            # unfolded outputs into the [batch, hidden_size * (numsteps-1)], and then reshape it into [batch * (numsteps-1), hidden_size]
            prediction_hidden_output = tf.reshape(tensor=tf.concat(values=prediction_target_hidden_output, axis=1), shape=[-1, self.hidden_size])

            # prediction has shape (batch * (numsteps - 1), self.input_dimension_size)
            prediction = tf.add(tf.matmul(prediction_hidden_output, self.W), self.bias, name='prediction')

           
            #with tf.compat.v1.variable_scope('Softmax_layer'):
            label_logits = self.dense1(last_cell)
                
                
           
        
        return label_logits
        return prediction, label_logits, last_cell

In [52]:
args = {
  'batch_size': 20,
  'epoch': 3,
  'lamda_D': 1.0,
  'G_epoch': 5,
  'train_data_filename': 'SITS',
  'test_data_filename': None,
  'layer_num': 1,
  'hidden_size': 100,
  'learning_rate': 0.001,
  'cell_type': 'GRU',
  'lamda': 1,
  'D_epoch': 1,
  'GPU': '0',
  'save': None,
  'verbose': 1,
  'num_steps': 54,
  'input_dimension_size': 16,
  'class_num': 8}

config = Config( **args)
config.batch_size

20

In [61]:
g = Generator(config)
#(20, 54, 16)

In [63]:
g(tf.ones((20, 54, 16)))

<tf.Tensor: shape=(20, 8), dtype=float32, numpy=
array([[ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.49603194,  0.10389157],
       [ 0.20953533,  0.19519186, -0.6723579 , -0.79262966,  0.01815289,
         0.25061285, -0.4960319

In [66]:
g.trainable_variables

[<tf.Variable 'Generator_LSTM/Project_W:0' shape=(100, 16) dtype=float32, numpy=
 array([[-0.16740161,  0.14539148, -0.12421192, ...,  0.05829823,
          0.19561909,  0.05258258],
        [ 0.06998283,  0.00174662, -0.06391335, ..., -0.04794777,
          0.10501041,  0.1325346 ],
        [-0.11309123, -0.07756921, -0.05660299, ...,  0.03700384,
         -0.07048105, -0.02705686],
        ...,
        [ 0.10210301, -0.0171556 ,  0.03799283, ..., -0.01746551,
          0.05699409,  0.00748228],
        [-0.04427364, -0.1631882 , -0.11639275, ...,  0.1528422 ,
         -0.00951483,  0.11049787],
        [ 0.02555117, -0.06583053,  0.03697964, ...,  0.19372034,
          0.01354282, -0.05677392]], dtype=float32)>,
 <tf.Variable 'Generator_LSTM/Project_bias:0' shape=(16,) dtype=float32, numpy=
 array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1], dtype=float32)>,
 <tf.Variable 'Generator_LSTM/RNN/stacked_rnn_cells_2/gru_cell_2/kernel:0' shape=(

In [93]:
d = keras.Sequential([layers.Input(20),layers.Dense(8)])

In [92]:
d(tf.ones((100, 20)))

ValueError: Input 0 of layer "sequential_6" is incompatible with the layer: expected shape=(None, 100), found shape=(100, 20)

In [112]:
logits = [[1.0, 1.0, 1.0]]
labels = [[10.0, 0.0, 0.0]]
tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([10.986122], dtype=float32)>

In [114]:
logits = tf.constant([1., -1., 0., 1., -1., 0., 0.])
labels = tf.constant([0., 0., 0., 1., 1., 1., 0.5])
tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits).numpy()



array([1.3132615 , 0.31326157, 0.6931472 , 0.31326157, 1.3132615 ,
       0.6931472 , 0.6931472 ], dtype=float32)